# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [19]:
import pandas as pd
import langchain_huggingface

In [13]:
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain



  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement LLMChain (from versions: none)
ERROR: No matching distribution found for LLMChain


In [15]:
%pip install -qU langchain-huggingface

In [3]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
  0% 0.00/3.73k [00:00<?, ?B/s]
100% 3.73k/3.73k [00:00<00:00, 6.40MB/s]
Archive:  dataset.zip
  inflating: Dataset.csv             


## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [4]:
import os
import markdown
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.embeddings.sentence_transformer import (SentenceTransformerEmbeddings,)
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
import json
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain import LLMChain

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


# Step 2: Load the Traffic Violations Dataset

In [5]:
df = pd.read_csv("/content/Dataset.csv")

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [6]:
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


In [7]:
df.shape

(104, 2)

In [8]:
df.isna().sum()

,0
المخالفة,0
الغرامة,0


In [9]:
df.duplicated().sum()

0

## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [20]:
directory='data/markdown_file'
os.makedirs(directory, exist_ok = True)

In [21]:
punctuations = '''`÷×؛<>_()*&^%][.ـ،/:"!?..,'{}~¦+|!”…“–ـ'''

In [28]:
for i in range(len(df)):
  title  = df['المخالفة'].iloc[i]
  content = df['الغرامة'].iloc[i]
  markdown_text = f'#{title}\n\n'
  markdown_text += f'{content}'

  with open(f'{directory}/ru{i}.md' , 'w' , encoding = 'utf-8') as file:
    file.write(markdown_text)


In [29]:
marked_text =[]
for fileName in os.listdir(directory):
  if fileName.endswith('.md'):
    with open(os.path.join(directory,fileName) , 'r' , encoding = 'utf-8') as file:
          temp = file.read()
          html_content = markdown.markdown(temp)
          marked_text.append(html_content)


In [30]:
marked_text

['<h1>الوقوف على خطوط السكة الحديدية.</h1>\n<p>الغرامة المالية 1000 - 2000 ريال</p>',
 '<h1>عدم ربط حزام الأمان.</h1>\n<p>الغرامة المالية 150 - 300 ريال</p>',
 '<h1>عدم الوقوف عند مراكز الضبط الأمني أو نقاط التفتيش أو عدم الوقوف للدورية الأمنية عند وجود توجيه أو علامة توجب الوقوف.</h1>\n<p>الغرامة المالية 3000 - 6000 ريال</p>',
 '<h1>سماح أصحاب الحيوانات بعبور حيواناتهم من غير الأماكن المخصصة لها، أو دون التنسيق مع الجهات المختصة.</h1>\n<p>الغرامة المالية 5000 - 10000 ريال</p>',
 '<h1>عدم إنهاء إجراءات تعديل مجال استعمال المركبة.</h1>\n<p>الغرامة المالية 1000 - 2000 ريال</p>',
 '<h1>سير المركبة بلا لوحة أمامية.</h1>\n<p>الغرامة المالية 1000 - 2000 ريال</p>',
 '<h1>القيادة على أكتاف الطريق وعلى الأرصفة أو المسارات التي يمنع القيادة فيها.</h1>\n<p>الغرامة المالية 1000 - 2000 ريال</p>',
 '<h1>عدم التقيد بتنظيمات السير عند تقاطعات الطرق.</h1>\n<p>الغرامة المالية 300 - 500 ريال</p>',
 '<h1>عدم قيام السائق في حال تغيير اتجاهه بالدوران للخلف بإعطاء أفضلية المرور للمركبات القادمة من الاتجاهات 

## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [32]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 50, chunk_overlap=10)
document = text_splitter.create_documents(marked_text)

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [33]:
embedding_function = SentenceTransformerEmbeddings(model_name="Omartificial-Intelligence-Space/GATE-AraBert-v0")
db = Chroma.from_documents(document, embedding_function, persist_directory="./chroma_db")

<ipython-input-33-edf67dddd28c>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding_function = SentenceTransformerEmbeddings(model_name="Omartificial-Intelligence-Space/GATE-AraBert-v0")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token 

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.50k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.02k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/761k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

# Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template encourages the model to give **advice** in **Arabic**, staying within the context provided.

In [35]:
PROMPT_TEMPLATE= '''
* الجواب فقط على السؤال المعطى
* التأكد من ان الاجابة توافق السؤال والسياق المعطى

السياق: {context}
السؤال: {question}
اجابتك هنا:
'''

prompt = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["context", "question"]
)

## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [36]:
my_API = "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"
llm = ChatGroq(temperature=0, groq_api_key=my_API, model_name="llama3-8b-8192")

## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [37]:
model = LLMChain(llm=llm, prompt=prompt, verbose=True)

<ipython-input-37-cc0327b2f172>:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  model = LLMChain(llm=llm, prompt=prompt, verbose=True)


## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [38]:
def query_rag(query: str):
  similarity_search_results = db.similarity_search_with_score(query, k=4)
  context_text = "\n\n".join([doc.page_content for doc, _score in similarity_search_results])#page content

  rag_respons = model.invoke({"context": context_text, "question": query})

  return rag_respons

## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي الغرامة على القيادة بدون رخصة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [39]:
response = query_rag("ماهي الغرامة على عدم وجود تأمين ساري للمركبة؟")
response

Prompt after formatting:

* الجواب فقط على السؤال المعطى
* التأكد من ان الاجابة توافق السؤال والسياق المعطى

السياق: <h1>عدم وجود تأمين ساري للمركبة.</h1>

<h1>عدم اتخاذ الاحتياطات اللازمة عند إيقاف المركبة

<h1>قيادة المركبة برخصة قيادة لا تتناسب مع حجم

<h1>تظليل زجاج المركبة دون التقيد بالضوابط التي
السؤال: ماهي الغرامة على عدم وجود تأمين ساري للمركبة؟
اجابتك هنا:


> Finished chain.


{'context': '<h1>عدم وجود تأمين ساري للمركبة.</h1>\n\n<h1>عدم اتخاذ الاحتياطات اللازمة عند إيقاف المركبة\n\n<h1>قيادة المركبة برخصة قيادة لا تتناسب مع حجم\n\n<h1>تظليل زجاج المركبة دون التقيد بالضوابط التي',
 'question': 'ماهي الغرامة على عدم وجود تأمين ساري للمركبة؟',
 'text': 'الغرامة على عدم وجود تأمين ساري للمركبة هي 500 ريال.'}

In [40]:
response = query_rag("ماهي الغرامة على ترك المركبة مفتوحة")
response

Prompt after formatting:

* الجواب فقط على السؤال المعطى
* التأكد من ان الاجابة توافق السؤال والسياق المعطى

السياق: <h1>قيادة المركبة برخصة قيادة لا تتناسب مع حجم

<h1>عدم إخراج المركبة المعدة للتصدير خلال المدة

<h1>استعمال المركبة لغير الغرض الذي رخصت من

<h1>عدم اتخاذ الاحتياطات اللازمة عند إيقاف المركبة
السؤال: ماهي الغرامة على ترك المركبة مفتوحة
اجابتك هنا:


> Finished chain.


{'context': '<h1>قيادة المركبة برخصة قيادة لا تتناسب مع حجم\n\n<h1>عدم إخراج المركبة المعدة للتصدير خلال المدة\n\n<h1>استعمال المركبة لغير الغرض الذي رخصت من\n\n<h1>عدم اتخاذ الاحتياطات اللازمة عند إيقاف المركبة',
 'question': 'ماهي الغرامة على ترك المركبة مفتوحة',
 'text': 'لا توجد غرامة على ترك المركبة مفتوحة، لأن السؤال يركز على الغرامة على ترك المركبة مفتوحة، وليس على تركها مفتوحة. السؤال يركز على الغرامة على مخالفات القيادة والمركبة، وليس على تركها مفتوحة.'}